<a href="https://colab.research.google.com/github/xVenve/POpenMP/blob/main/POpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#POpenMP

1. Run Create files
2. Insert the .bmp images in the input folder.
3. Drop down seq or par to run the filter of your choice.
4. The filtered images will be in output

# Create files

In [26]:
%%writefile image-seq.cpp
#include <chrono>
#include <cmath>
#include <cstring>
#include <dirent.h>
#include <fstream>
#include <iostream>
#include <stdlib.h>
using namespace std;
using namespace std::chrono;
using clk = chrono::high_resolution_clock;

void gauss(int, int, unsigned char *, unsigned char *);

int main(int argc, char **argv) {

  // Controlar el numero de parametros
  if (argc != 4) {
    cout << "Wrong format:\n " << argv[0]
         << " operation in_path out_path\n   "
            "operation: copy, gauss, sobel\n";
    return -1;
  }

  // Comprobar que la operacion sea una de las admitidas
  if (strcmp(argv[1], "gauss") && strcmp(argv[1], "copy") &&
      strcmp(argv[1], "sobel")) {
    cout << "Unexpected operation:" << argv[1] << "\n"
         << argv[0]
         << " operation in_path out_path\n   operation: copy, "
            "gauss, sobel\n";
    return -1;
  }

  // Comprobar directorio de entrada existente
  DIR *input = opendir(argv[2]);
  if (input == NULL) {
    cout << "Input path: " << argv[2] << "\nOutput path: " << argv[3]
         << "\nCannot open directory [" << argv[2] << "]"
         << "\n"
         << argv[0]
         << " operation in_path out_path\n   operation: copy, "
            "gauss, sobel\n";
    return -1;
  }

  // Comprobar directorio de salida existente
  DIR *output = opendir(argv[3]);
  if (output == NULL) {
    cout << "Input path: " << argv[2] << "\nOutput path: " << argv[3]
         << "\nOutput directory [" << argv[3] << "] does not exist"
         << "\n"
         << argv[0]
         << " operation in_path out_path\n   operation: copy, "
            "gauss, sobel\n";
    return -1;
  }

  cout << "Input path: " << argv[2] << "\nOutput path: " << argv[3] << "\n";

  struct dirent *dir;
  while ((dir = readdir(input)) != NULL) {
    if (strcmp(dir->d_name, ".") != 0 && strcmp(dir->d_name, "..") != 0) {

      char *nameinput = (char *)malloc(strlen(argv[2]) + strlen("/") +
                                       strlen(dir->d_name) + 1);
      strcpy(nameinput, argv[2]);
      strcat(nameinput, "/");
      strcat(nameinput, dir->d_name);

      char *nameoutput = (char *)malloc(strlen(argv[3]) + strlen("/") +
                                        strlen(dir->d_name) + 1);
      strcpy(nameoutput, argv[3]);
      strcat(nameoutput, "/");
      strcat(nameoutput, dir->d_name);

      string name = dir->d_name;

      // Comprobación de formato
      int position = name.find_last_of(".");
      string extension = name.substr(position + 1);
      int result = extension.compare("bmp");
      if (result != 0) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << " Unexpected file without .bmp extension: " << name << "\n";
        continue;
      }

      auto loadtimei = clk ::now();
      FILE *photo = fopen(nameinput, "rb");
      if (photo == NULL)
        cout << "Error processing \"" << nameinput << "\" \n"
             << "Error al abrir el fichero." << '\n';

      // Cabecera
      unsigned char *info = new unsigned char[54];
      int freaderror = fread(info, sizeof(unsigned char), 54, photo);
      if (freaderror < 0) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << "ERROR al leer la cabecera de la imagen" << '\n';
        continue;
      }

      // Ancho y alto
      int width = *(int *)&info[18];
      int height = *(int *)&info[22];

      // 24 bits = 3 bytes por pixel
      int size = *(int *)&info[2] - *(int *)&info[10];
      unsigned char *data = new unsigned char[size - height * (width % 4)];

      // Error de .bmp que no tenga un plano
      if (*(unsigned short *)&info[26] != 1) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << "Planes is not 1\n " << argv[0]
             << " operation in_path out_path\n   operation: copy, "
                "gauss, sobel\n";
        continue;
      }

      // Error de .bmp que no sea 24 bits por punto
      if (*(unsigned short *)&info[28] != 24) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << "Bit count is not 24.\n " << argv[0]
             << " operation in_path out_path\n   operation: copy, "
                "gauss, sobel\n";
        continue;
      }

      // Error de .bmp que no sea 0 la compresion
      if (*(int *)&info[30] != 0) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << "Compression is not 0.\n " << argv[0]
             << " operation in_path out_path\n   operation: copy, "
                "gauss, sobel\n";
        continue;
      }

      // Lectura de los datos de imagen
      fseek(photo, *(int *)&info[10], SEEK_SET);
      if (width % 4 == 0) {
        freaderror = fread(data, sizeof(unsigned char), size, photo);
      } else {
        for (int i = 0; i < height && freaderror >= 0; i++) {
          freaderror = fread(&data[i * width * 3], sizeof(unsigned char),
                             width * 3, photo);
          fseek(photo, (width % 4), SEEK_CUR);
        }
      }

      if (freaderror < 0) {
        cout << "ERROR al leer el cuerpo de la imagen" << '\n';
        return -1;
      }
      auto loadtimef = clk ::now();

      // FUNCION COPY
      if (strcmp(argv[1], "copy") == 0) {
        ofstream foutput;

        auto storetimei = clk ::now();
        foutput.open(nameoutput);
        // creacion de la cabecera
        foutput.write("B", 1);
        foutput.write("M", 1);
        int tamanoarchivo = size + 54;
        foutput.write(reinterpret_cast<const char *>(&tamanoarchivo), 4);
        int cero = 0;
        int cincocuatro = 54;
        int cuarenta = 40;
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cincocuatro), 4);
        foutput.write(reinterpret_cast<const char *>(&cuarenta), 4);
        foutput.write(reinterpret_cast<const char *>(&width), 4);
        foutput.write(reinterpret_cast<const char *>(&height), 4);
        int uno = 1;
        foutput.write(reinterpret_cast<const char *>(&uno), 2);
        int veinticuatro = 24;
        foutput.write(reinterpret_cast<const char *>(&veinticuatro), 2);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&size), 4);
        int resolucion = 2835;
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);

        // Creacion de datos imagen
        unsigned char c = 0;
        if (width % 4 == 0) {
          foutput.write(reinterpret_cast<const char *>(data), size);
        } else {
          for (int i = 0; i < height; i++) {
            foutput.write(reinterpret_cast<const char *>(&data[3 * i * width]),
                          width * 3);
            if (width % 4 != 0) {
              foutput.write(reinterpret_cast<const char *>(&c), width % 4);
            }
          }
        }

        auto storetimef = clk ::now();
        foutput.close();

        auto loaddiff = duration_cast<microseconds>(loadtimef - loadtimei);
        auto storediff = duration_cast<microseconds>(storetimef - storetimei);
        auto sum = loaddiff + storediff;

        cout << "File: \"" << nameinput << "\"(time: " << sum.count() << ")\n";
        cout << "  Load time: " << loaddiff.count() << "\n";
        cout << "  Store time: " << storediff.count() << "\n";
      }

      // FUNCION GAUSS
      if (strcmp(argv[1], "gauss") == 0) {

        unsigned char *res = new unsigned char[size - height * (width % 4)];
        auto gausstimei = clk ::now();
        gauss(width, height, data, res);
        auto gausstimef = clk ::now();

        auto storetimei = clk ::now();
        ofstream foutput;
        foutput.open(nameoutput);

        // creacion de la cabezera
        foutput.write("B", 1);
        foutput.write("M", 1);
        int tamanoarchivo = size + 54;
        foutput.write(reinterpret_cast<const char *>(&tamanoarchivo), 4);
        int cero = 0;
        int cincocuatro = 54;
        int cuarenta = 40;
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cincocuatro), 4);
        foutput.write(reinterpret_cast<const char *>(&cuarenta), 4);
        foutput.write(reinterpret_cast<const char *>(&width), 4);
        foutput.write(reinterpret_cast<const char *>(&height), 4);
        int uno = 1;
        foutput.write(reinterpret_cast<const char *>(&uno), 2);
        int veinticuatro = 24;
        foutput.write(reinterpret_cast<const char *>(&veinticuatro), 2);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&size), 4);
        int resolucion = 2835;
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        // Creacion de datos imagen
        unsigned char c = 0;
        if (width % 4 == 0) {
          foutput.write(reinterpret_cast<const char *>(res), size);
        } else {
          for (int i = 0; i < height; i++) {
            foutput.write(reinterpret_cast<const char *>(&res[3 * i * width]),
                          width * 3);
            if (width % 4 != 0) {
              foutput.write(reinterpret_cast<const char *>(&c), width % 4);
            }
          }
        }
        auto storetimef = clk ::now();
        foutput.close();

        auto loaddiff = duration_cast<microseconds>(loadtimef - loadtimei);
        auto gaussdiff = duration_cast<microseconds>(gausstimef - gausstimei);
        auto storediff = duration_cast<microseconds>(storetimef - storetimei);
        auto sum = loaddiff + storediff + gaussdiff;

        cout << "File: \"" << nameinput << "\"(time: " << sum.count() << ")\n";
        cout << "  Load time: " << loaddiff.count() << "\n";
        cout << "  Gauss time: " << gaussdiff.count() << "\n";
        cout << "  Store time: " << storediff.count() << "\n";
        free(res);
      }

      // FUNCION SOBEL
      if (strcmp(argv[1], "sobel") == 0) {
        unsigned char *gaussres =
            new unsigned char[size - height * (width % 4)];

        auto gausstimei = clk ::now();
        gauss(width, height, data, gaussres);
        auto gausstimef = clk ::now();

        int mx[3][3] = {{1, 2, 1}, {0, 0, 0}, {-1, -2, -1}};

        int my[3][3] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};

        unsigned char *sobel = new unsigned char[size - height * (width % 4)];
        auto sobeltimei = clk ::now();
        for (int i = 0; i < height; i++) {
          for (int j = 0; j < width; j++) {
            int xx = 0;
            int xy = 0;
            int xz = 0;

            int yx = 0;
            int yy = 0;
            int yz = 0;
            for (int s = -1; s < 2; s++) {
              for (int t = -1; t < 2; t++) {
                // Condición para marcado de bordes
                if ((i + s) < height && (j + t) < width && (i + s) >= 0 &&
                    (j + t) >= 0) {
                  xx += mx[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t))];
                  xy += mx[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t)) + 1];
                  xz += mx[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t)) + 2];

                  yx += my[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t))];
                  yy += my[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t)) + 1];
                  yz += my[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t)) + 2];
                }
              }
            }
            sobel[3 * (i * width + j)] = (abs(xx) + abs(yx)) / 8;
            sobel[3 * (i * width + j) + 1] = (abs(xy) + abs(yy)) / 8;
            sobel[3 * (i * width + j) + 2] = (abs(xz) + abs(yz)) / 8;
          }
        }
        auto sobeltimef = clk ::now();

        auto storetimei = clk ::now();
        ofstream foutput;
        foutput.open(nameoutput);
        // creacion de la cabecera
        foutput.write("B", 1);
        foutput.write("M", 1);
        int tamanoarchivo = size + 54;
        foutput.write(reinterpret_cast<const char *>(&tamanoarchivo), 4);
        int cero = 0;
        int cincocuatro = 54;
        int cuarenta = 40;
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cincocuatro), 4);
        foutput.write(reinterpret_cast<const char *>(&cuarenta), 4);
        foutput.write(reinterpret_cast<const char *>(&width), 4);
        foutput.write(reinterpret_cast<const char *>(&height), 4);
        int uno = 1;
        foutput.write(reinterpret_cast<const char *>(&uno), 2);
        int veinticuatro = 24;
        foutput.write(reinterpret_cast<const char *>(&veinticuatro), 2);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&size), 4);
        int resolucion = 2835;
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);

        // Creacion de datos imagen
        unsigned char c = 0;
        if (width % 4 == 0) {
          foutput.write(reinterpret_cast<const char *>(sobel), size);
        } else {
          for (int i = 0; i < height; i++) {
            foutput.write(reinterpret_cast<const char *>(&sobel[3 * i * width]),
                          width * 3);
            if (width % 4 != 0) {
              foutput.write(reinterpret_cast<const char *>(&c), width % 4);
            }
          }
        }
        auto storetimef = clk ::now();
        foutput.close();

        auto loaddiff = duration_cast<microseconds>(loadtimef - loadtimei);
        auto gaussdiff = duration_cast<microseconds>(gausstimef - gausstimei);
        auto sobeldiff = duration_cast<microseconds>(sobeltimef - sobeltimei);
        auto storediff = duration_cast<microseconds>(storetimef - storetimei);
        auto sum = loaddiff + gaussdiff + storediff + sobeldiff;

        cout << "File: \"" << nameinput << "\"(time: " << sum.count() << ")\n";
        cout << "  Load time: " << loaddiff.count() << "\n";
        cout << "  Gauss time: " << gaussdiff.count() << "\n";
        cout << "  Sobel time: " << sobeldiff.count() << "\n";
        cout << "  Store time: " << storediff.count() << "\n";
        free(gaussres);
        free(sobel);
      }

      free(nameinput);
      free(data);
      free(info);
      free(nameoutput);
      fclose(photo);
    }
  }

  closedir(input);
  closedir(output);
  return 0;
}

void gauss(int width, int height, unsigned char *data, unsigned char *res) {
  int m[5][5] = {{1, 4, 7, 4, 1},
                 {4, 16, 26, 16, 4},
                 {7, 26, 41, 26, 7},
                 {4, 16, 26, 16, 4},
                 {1, 4, 7, 4, 1}};

  for (int i = 0; i < height; i++) {
    for (int j = 0; j < width; j++) {
      int x = 0;
      int y = 0;
      int z = 0;
      for (int s = -2; s < 3; s++) {
        for (int t = -2; t < 3; t++) {
          // Condición para marcado de bordes
          if ((i + s) < height && (j + t) < width && (i + s) >= 0 &&
              (j + t) >= 0) {
            x += m[s + 2][t + 2] * data[3 * ((i + s) * width + (j + t))];
            y += m[s + 2][t + 2] * data[3 * ((i + s) * width + (j + t)) + 1];
            z += m[s + 2][t + 2] * data[3 * ((i + s) * width + (j + t)) + 2];
          }
        }
      }
      res[3 * (i * width + j)] = x / 273;
      res[3 * (i * width + j) + 1] = y / 273;
      res[3 * (i * width + j) + 2] = z / 273;
    }
  }
}

Overwriting image-seq.cpp


In [27]:
%%writefile image-par.cpp
#include <chrono>
#include <cmath>
#include <cstring>
#include <dirent.h>
#include <fstream>
#include <iostream>
#include <omp.h>
#include <stdlib.h>
using namespace std;
using namespace std::chrono;
using clk = chrono::high_resolution_clock;

void gauss(int, int, unsigned char *, unsigned char *);

int main(int argc, char **argv) {

  // Controlar el numero de parametros
  if (argc != 4) {
    cout << "Wrong format:\n " << argv[0]
         << " operation in_path out_path\n   "
            "operation: copy, gauss, sobel\n";
    return -1;
  }

  // Comprobar que la operacion sea una de las admitidas
  if (strcmp(argv[1], "gauss") && strcmp(argv[1], "copy") &&
      strcmp(argv[1], "sobel")) {
    cout << "Unexpected operation:" << argv[1] << "\n"
         << argv[0]
         << " operation in_path out_path\n   operation: copy, "
            "gauss, sobel\n";
    return -1;
  }

  // Comprobar directorio de entrada existente
  DIR *input = opendir(argv[2]);
  if (input == NULL) {
    cout << "Input path: " << argv[2] << "\nOutput path: " << argv[3]
         << "\nCannot open directory [" << argv[2] << "]"
         << "\n"
         << argv[0]
         << " operation in_path out_path\n   operation: copy, "
            "gauss, sobel\n";
    return -1;
  }

  // Comprobar directorio de salida existente
  DIR *output = opendir(argv[3]);
  if (output == NULL) {
    cout << "Input path: " << argv[2] << "\nOutput path: " << argv[3]
         << "\nOutput directory [" << argv[3] << "] does not exist"
         << "\n"
         << argv[0]
         << " operation in_path out_path\n   operation: copy, "
            "gauss, sobel\n";
    return -1;
  }

  cout << "Input path: " << argv[2] << "\nOutput path: " << argv[3] << "\n";

  struct dirent *dir;
  while ((dir = readdir(input)) != NULL) {
    if (strcmp(dir->d_name, ".") != 0 && strcmp(dir->d_name, "..") != 0) {

      char *nameinput = (char *)malloc(strlen(argv[2]) + strlen("/") +
                                       strlen(dir->d_name) + 1);
      strcpy(nameinput, argv[2]);
      strcat(nameinput, "/");
      strcat(nameinput, dir->d_name);

      char *nameoutput = (char *)malloc(strlen(argv[3]) + strlen("/") +
                                        strlen(dir->d_name) + 1);
      strcpy(nameoutput, argv[3]);
      strcat(nameoutput, "/");
      strcat(nameoutput, dir->d_name);

      string name = dir->d_name;

      // Comprobación de formato
      int position = name.find_last_of(".");
      string extension = name.substr(position + 1);
      int result = extension.compare("bmp");
      if (result != 0) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << " Unexpected file without .bmp extension: " << name << "\n";
        continue;
      }

      auto loadtimei = clk ::now();
      FILE *photo = fopen(nameinput, "rb");
      if (photo == NULL)
        cout << "Error processing \"" << nameinput << "\" \n"
             << "Error al abrir el fichero." << '\n';

      // Cabecera
      unsigned char *info = new unsigned char[54];
      int freaderror = fread(info, sizeof(unsigned char), 54, photo);
      if (freaderror < 0) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << "ERROR al leer la cabecera de la imagen" << '\n';
        continue;
      }

      // Ancho y alto
      int width = *(int *)&info[18];
      int height = *(int *)&info[22];

      // 24 bits = 3 bytes por pixel
      int size = *(int *)&info[2] - *(int *)&info[10];
      unsigned char *data = new unsigned char[size - height * (width % 4)];

      // Error de .bmp que no tenga un plano
      if (*(unsigned short *)&info[26] != 1) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << "Planes is not 1\n " << argv[0]
             << " operation in_path out_path\n   operation: copy, "
                "gauss, sobel\n";
        continue;
      }

      // Error de .bmp que no sea 24 bits por punto
      if (*(unsigned short *)&info[28] != 24) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << "Bit count is not 24.\n " << argv[0]
             << " operation in_path out_path\n   operation: copy, "
                "gauss, sobel\n";
        continue;
      }

      // Error de .bmp que no sea 0 la compresion
      if (*(int *)&info[30] != 0) {
        cout << "Error processing \"" << nameinput << "\" \n"
             << "Compression is not 0.\n " << argv[0]
             << " operation in_path out_path\n   operation: copy, "
                "gauss, sobel\n";
        continue;
      }

      // Lectura de los datos de imagen
      fseek(photo, *(int *)&info[10], SEEK_SET);
      if (width % 4 == 0) {
        freaderror = fread(data, sizeof(unsigned char), size, photo);
      } else {
        for (int i = 0; i < height && freaderror >= 0; i++) {
          freaderror = fread(&data[i * width * 3], sizeof(unsigned char),
                             width * 3, photo);
          fseek(photo, (width % 4), SEEK_CUR);
        }
      }

      if (freaderror < 0) {
        cout << "ERROR al leer el cuerpo de la imagen" << '\n';
        return -1;
      }
      auto loadtimef = clk ::now();

      // FUNCION COPY
      if (strcmp(argv[1], "copy") == 0) {
        ofstream foutput;

        auto storetimei = clk ::now();
        foutput.open(nameoutput);
        // creacion de la cabecera
        foutput.write("B", 1);
        foutput.write("M", 1);
        int tamanoarchivo = size + 54;
        foutput.write(reinterpret_cast<const char *>(&tamanoarchivo), 4);
        int cero = 0;
        int cincocuatro = 54;
        int cuarenta = 40;
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cincocuatro), 4);
        foutput.write(reinterpret_cast<const char *>(&cuarenta), 4);
        foutput.write(reinterpret_cast<const char *>(&width), 4);
        foutput.write(reinterpret_cast<const char *>(&height), 4);
        int uno = 1;
        foutput.write(reinterpret_cast<const char *>(&uno), 2);
        int veinticuatro = 24;
        foutput.write(reinterpret_cast<const char *>(&veinticuatro), 2);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&size), 4);
        int resolucion = 2835;
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);

        // Creacion de datos imagen
        unsigned char c = 0;
        if (width % 4 == 0) {
          foutput.write(reinterpret_cast<const char *>(data), size);
        } else {
          for (int i = 0; i < height; i++) {
            foutput.write(reinterpret_cast<const char *>(&data[3 * i * width]),
                          width * 3);
            if (width % 4 != 0) {
              foutput.write(reinterpret_cast<const char *>(&c), width % 4);
            }
          }
        }

        auto storetimef = clk ::now();
        foutput.close();

        auto loaddiff = duration_cast<microseconds>(loadtimef - loadtimei);
        auto storediff = duration_cast<microseconds>(storetimef - storetimei);
        auto sum = loaddiff + storediff;

        cout << "File: \"" << nameinput << "\"(time: " << sum.count() << ")\n";
        cout << "  Load time: " << loaddiff.count() << "\n";
        cout << "  Store time: " << storediff.count() << "\n";
      }

      // FUNCION GAUSS
      if (strcmp(argv[1], "gauss") == 0) {

        unsigned char *res = new unsigned char[size - height * (width % 4)];
        auto gausstimei = clk ::now();
        gauss(width, height, data, res);
        auto gausstimef = clk ::now();

        auto storetimei = clk ::now();
        ofstream foutput;
        foutput.open(nameoutput);

        // creacion de la cabezera
        foutput.write("B", 1);
        foutput.write("M", 1);
        int tamanoarchivo = size + 54;
        foutput.write(reinterpret_cast<const char *>(&tamanoarchivo), 4);
        int cero = 0;
        int cincocuatro = 54;
        int cuarenta = 40;
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cincocuatro), 4);
        foutput.write(reinterpret_cast<const char *>(&cuarenta), 4);
        foutput.write(reinterpret_cast<const char *>(&width), 4);
        foutput.write(reinterpret_cast<const char *>(&height), 4);
        int uno = 1;
        foutput.write(reinterpret_cast<const char *>(&uno), 2);
        int veinticuatro = 24;
        foutput.write(reinterpret_cast<const char *>(&veinticuatro), 2);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&size), 4);
        int resolucion = 2835;
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        // Creacion de datos imagen
        unsigned char c = 0;
        if (width % 4 == 0) {
          foutput.write(reinterpret_cast<const char *>(res), size);
        } else {
          for (int i = 0; i < height; i++) {
            foutput.write(reinterpret_cast<const char *>(&res[3 * i * width]),
                          width * 3);
            if (width % 4 != 0) {
              foutput.write(reinterpret_cast<const char *>(&c), width % 4);
            }
          }
        }
        auto storetimef = clk ::now();
        foutput.close();

        auto loaddiff = duration_cast<microseconds>(loadtimef - loadtimei);
        auto gaussdiff = duration_cast<microseconds>(gausstimef - gausstimei);
        auto storediff = duration_cast<microseconds>(storetimef - storetimei);
        auto sum = loaddiff + storediff + gaussdiff;

        cout << "File: \"" << nameinput << "\"(time: " << sum.count() << ")\n";
        cout << "  Load time: " << loaddiff.count() << "\n";
        cout << "  Gauss time: " << gaussdiff.count() << "\n";
        cout << "  Store time: " << storediff.count() << "\n";
        free(res);
      }

      // FUNCION SOBEL
      if (strcmp(argv[1], "sobel") == 0) {
        unsigned char *gaussres =
            new unsigned char[size - height * (width % 4)];

        auto gausstimei = clk ::now();
        gauss(width, height, data, gaussres);
        auto gausstimef = clk ::now();

        int mx[3][3] = {{1, 2, 1}, {0, 0, 0}, {-1, -2, -1}};

        int my[3][3] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};

        unsigned char *sobel = new unsigned char[size - height * (width % 4)];
        auto sobeltimei = clk ::now();
#pragma omp parallel for num_threads(8) schedule(static)
        for (int i = 0; i < height; i++) {
          for (int j = 0; j < width; j++) {
            int xx = 0;
            int xy = 0;
            int xz = 0;

            int yx = 0;
            int yy = 0;
            int yz = 0;
            for (int s = -1; s < 2; s++) {
              for (int t = -1; t < 2; t++) {
                // Condición para marcado de bordes
                if ((i + s) < height && (j + t) < width && (i + s) >= 0 &&
                    (j + t) >= 0) {
                  xx += mx[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t))];
                  xy += mx[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t)) + 1];
                  xz += mx[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t)) + 2];

                  yx += my[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t))];
                  yy += my[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t)) + 1];
                  yz += my[s + 1][t + 1] *
                        gaussres[3 * ((i + s) * width + (j + t)) + 2];
                }
              }
            }
            sobel[3 * (i * width + j)] = (abs(xx) + abs(yx)) / 8;
            sobel[3 * (i * width + j) + 1] = (abs(xy) + abs(yy)) / 8;
            sobel[3 * (i * width + j) + 2] = (abs(xz) + abs(yz)) / 8;
          }
        }
        auto sobeltimef = clk ::now();

        auto storetimei = clk ::now();
        ofstream foutput;
        foutput.open(nameoutput);
        // creacion de la cabecera
        foutput.write("B", 1);
        foutput.write("M", 1);
        int tamanoarchivo = size + 54;
        foutput.write(reinterpret_cast<const char *>(&tamanoarchivo), 4);
        int cero = 0;
        int cincocuatro = 54;
        int cuarenta = 40;
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cincocuatro), 4);
        foutput.write(reinterpret_cast<const char *>(&cuarenta), 4);
        foutput.write(reinterpret_cast<const char *>(&width), 4);
        foutput.write(reinterpret_cast<const char *>(&height), 4);
        int uno = 1;
        foutput.write(reinterpret_cast<const char *>(&uno), 2);
        int veinticuatro = 24;
        foutput.write(reinterpret_cast<const char *>(&veinticuatro), 2);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&size), 4);
        int resolucion = 2835;
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&resolucion), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);
        foutput.write(reinterpret_cast<const char *>(&cero), 4);

        // Creacion de datos imagen
        unsigned char c = 0;
        if (width % 4 == 0) {
          foutput.write(reinterpret_cast<const char *>(sobel), size);
        } else {
          for (int i = 0; i < height; i++) {
            foutput.write(reinterpret_cast<const char *>(&sobel[3 * i * width]),
                          width * 3);
            if (width % 4 != 0) {
              foutput.write(reinterpret_cast<const char *>(&c), width % 4);
            }
          }
        }
        auto storetimef = clk ::now();
        foutput.close();

        auto loaddiff = duration_cast<microseconds>(loadtimef - loadtimei);
        auto gaussdiff = duration_cast<microseconds>(gausstimef - gausstimei);
        auto sobeldiff = duration_cast<microseconds>(sobeltimef - sobeltimei);
        auto storediff = duration_cast<microseconds>(storetimef - storetimei);
        auto sum = loaddiff + gaussdiff + storediff + sobeldiff;

        cout << "File: \"" << nameinput << "\"(time: " << sum.count() << ")\n";
        cout << "  Load time: " << loaddiff.count() << "\n";
        cout << "  Gauss time: " << gaussdiff.count() << "\n";
        cout << "  Sobel time: " << sobeldiff.count() << "\n";
        cout << "  Store time: " << storediff.count() << "\n";
        free(gaussres);
        free(sobel);
      }

      free(nameinput);
      free(data);
      free(info);
      free(nameoutput);
      fclose(photo);
    }
  }

  closedir(input);
  closedir(output);
  return 0;
}

void gauss(int width, int height, unsigned char *data, unsigned char *res) {
  int m[5][5] = {{1, 4, 7, 4, 1},
                 {4, 16, 26, 16, 4},
                 {7, 26, 41, 26, 7},
                 {4, 16, 26, 16, 4},
                 {1, 4, 7, 4, 1}};
#pragma omp parallel for num_threads(8) schedule(dynamic)
  for (int i = 0; i < height; i++) {
    for (int j = 0; j < width; j++) {
      int x = 0;
      int y = 0;
      int z = 0;
      for (int s = -2; s < 3; s++) {
        for (int t = -2; t < 3; t++) {
          // Condición para marcado de bordes
          if ((i + s) < height && (j + t) < width && (i + s) >= 0 &&
              (j + t) >= 0) {
            x += m[s + 2][t + 2] * data[3 * ((i + s) * width + (j + t))];
            y += m[s + 2][t + 2] * data[3 * ((i + s) * width + (j + t)) + 1];
            z += m[s + 2][t + 2] * data[3 * ((i + s) * width + (j + t)) + 2];
          }
        }
      }
      res[3 * (i * width + j)] = x / 273;
      res[3 * (i * width + j) + 1] = y / 273;
      res[3 * (i * width + j) + 2] = z / 273;
    }
  }
}

Overwriting image-par.cpp


In [28]:
%%writefile Makefile
CC=g++
CFLAGS=-std=c++17 -Wall -Wextra -Wno-deprecated -Werror -pedantic -pedantic-errors -O3 -DNDEBUG
PARFLAGS=-fopenmp

all: image-seq image-par

image-seq:	image-seq.cpp
	$(CC) $(CFLAGS) $< -o $@

image-par:	image-par.cpp
	$(CC) $(CFLAGS) $(PARFLAGS) $< -o $@

Overwriting Makefile


# Execution seq


Copy

In [37]:
%%shell
make image-seq 
./image-seq copy /content/input /content/output

make: 'image-seq' is up to date.
Input path: /content/input
Output path: /content/output
File: "/content/input/car.bmp"(time: 260)
  Load time: 105
  Store time: 155
File: "/content/input/maze.bmp"(time: 4251)
  Load time: 1236
  Store time: 3015
File: "/content/input/balloon.bmp"(time: 436)
  Load time: 171
  Store time: 265
File: "/content/input/car2.bmp"(time: 3493)
  Load time: 1107
  Store time: 2386
File: "/content/input/elephant.bmp"(time: 3123)
  Load time: 739
  Store time: 2384
File: "/content/input/flower.bmp"(time: 1977)
  Load time: 733
  Store time: 1244
File: "/content/input/bath.bmp"(time: 1084)
  Load time: 236
  Store time: 848
File: "/content/input/pyramid.bmp"(time: 12835)
  Load time: 3403
  Store time: 9432
File: "/content/input/amphora.bmp"(time: 7903)
  Load time: 1625
  Store time: 6278
File: "/content/input/tiger.bmp"(time: 3910)
  Load time: 356
  Store time: 3554
File: "/content/input/plane.bmp"(time: 2414)
  Load time: 653
  Store time: 1761
File: "/content

Gauss

In [39]:
%%shell
make image-seq 
./image-seq gauss /content/input /content/output

make: 'image-seq' is up to date.
Input path: /content/input
Output path: /content/output
File: "/content/input/car.bmp"(time: 1199)
  Load time: 87
  Gauss time: 931
  Store time: 181
File: "/content/input/maze.bmp"(time: 22327)
  Load time: 1268
  Gauss time: 18701
  Store time: 2358
File: "/content/input/balloon.bmp"(time: 1970)
  Load time: 191
  Gauss time: 1559
  Store time: 220
File: "/content/input/car2.bmp"(time: 45364)
  Load time: 1048
  Gauss time: 41925
  Store time: 2391
File: "/content/input/elephant.bmp"(time: 45870)
  Load time: 1443
  Gauss time: 41976
  Store time: 2451
File: "/content/input/flower.bmp"(time: 20881)
  Load time: 770
  Gauss time: 18905
  Store time: 1206
File: "/content/input/bath.bmp"(time: 14352)
  Load time: 490
  Gauss time: 13011
  Store time: 851
File: "/content/input/pyramid.bmp"(time: 40236)
  Load time: 994
  Gauss time: 36980
  Store time: 2262
File: "/content/input/amphora.bmp"(time: 44956)
  Load time: 1595
  Gauss time: 38589
  Store time

Sobel


In [40]:
%%shell
make image-seq 
./image-seq sobel /content/input /content/output

make: 'image-seq' is up to date.
Input path: /content/input
Output path: /content/output
File: "/content/input/car.bmp"(time: 2268)
  Load time: 84
  Gauss time: 1309
  Sobel time: 668
  Store time: 207
File: "/content/input/maze.bmp"(time: 33516)
  Load time: 1546
  Gauss time: 20025
  Sobel time: 9441
  Store time: 2504
File: "/content/input/balloon.bmp"(time: 2568)
  Load time: 179
  Gauss time: 1501
  Sobel time: 699
  Store time: 189
File: "/content/input/car2.bmp"(time: 61537)
  Load time: 854
  Gauss time: 36520
  Sobel time: 22171
  Store time: 1992
File: "/content/input/elephant.bmp"(time: 56650)
  Load time: 1037
  Gauss time: 35182
  Sobel time: 18468
  Store time: 1963
File: "/content/input/flower.bmp"(time: 30458)
  Load time: 565
  Gauss time: 18226
  Sobel time: 10435
  Store time: 1232
File: "/content/input/bath.bmp"(time: 20945)
  Load time: 363
  Gauss time: 13192
  Sobel time: 6660
  Store time: 730
File: "/content/input/pyramid.bmp"(time: 57790)
  Load time: 814
  G

# Execution par

Copy


In [41]:
%%shell
make image-par 
./image-par copy /content/input /content/output

make: 'image-par' is up to date.
Input path: /content/input
Output path: /content/output
File: "/content/input/car.bmp"(time: 213)
  Load time: 57
  Store time: 156
File: "/content/input/maze.bmp"(time: 2929)
  Load time: 966
  Store time: 1963
File: "/content/input/balloon.bmp"(time: 275)
  Load time: 106
  Store time: 169
File: "/content/input/car2.bmp"(time: 2642)
  Load time: 923
  Store time: 1719
File: "/content/input/elephant.bmp"(time: 2294)
  Load time: 554
  Store time: 1740
File: "/content/input/flower.bmp"(time: 1835)
  Load time: 656
  Store time: 1179
File: "/content/input/bath.bmp"(time: 1096)
  Load time: 228
  Store time: 868
File: "/content/input/pyramid.bmp"(time: 2337)
  Load time: 572
  Store time: 1765
File: "/content/input/amphora.bmp"(time: 5088)
  Load time: 1099
  Store time: 3989
File: "/content/input/tiger.bmp"(time: 988)
  Load time: 272
  Store time: 716
File: "/content/input/plane.bmp"(time: 2265)
  Load time: 522
  Store time: 1743
File: "/content/input/

Gauss


In [33]:
%%shell
make image-par 
./image-par gauss /content/input /content/output

make: 'image-par' is up to date.
Input path: /content/input
Output path: /content/output
File: "/content/input/car.bmp"(time: 1566)
  Load time: 77
  Gauss time: 1320
  Store time: 169
File: "/content/input/maze.bmp"(time: 22499)
  Load time: 1161
  Gauss time: 18945
  Store time: 2393
File: "/content/input/balloon.bmp"(time: 2127)
  Load time: 210
  Gauss time: 1712
  Store time: 205
File: "/content/input/car2.bmp"(time: 42869)
  Load time: 1068
  Gauss time: 39509
  Store time: 2292
File: "/content/input/elephant.bmp"(time: 48532)
  Load time: 1465
  Gauss time: 44798
  Store time: 2269
File: "/content/input/flower.bmp"(time: 22206)
  Load time: 774
  Gauss time: 20196
  Store time: 1236
File: "/content/input/bath.bmp"(time: 15383)
  Load time: 458
  Gauss time: 14063
  Store time: 862
File: "/content/input/pyramid.bmp"(time: 42618)
  Load time: 1008
  Gauss time: 39287
  Store time: 2323
File: "/content/input/amphora.bmp"(time: 46135)
  Load time: 1739
  Gauss time: 38459
  Store ti

Sobel


In [34]:
%%shell
make image-par 
./image-par sobel /content/input /content/output

make: 'image-par' is up to date.
Input path: /content/input
Output path: /content/output
File: "/content/input/car.bmp"(time: 2153)
  Load time: 79
  Gauss time: 1252
  Sobel time: 664
  Store time: 158
File: "/content/input/maze.bmp"(time: 34334)
  Load time: 1057
  Gauss time: 19017
  Sobel time: 12111
  Store time: 2149
File: "/content/input/balloon.bmp"(time: 2995)
  Load time: 228
  Gauss time: 1605
  Sobel time: 958
  Store time: 204
File: "/content/input/car2.bmp"(time: 69734)
  Load time: 850
  Gauss time: 39311
  Sobel time: 27563
  Store time: 2010
File: "/content/input/elephant.bmp"(time: 64854)
  Load time: 1062
  Gauss time: 38497
  Sobel time: 23340
  Store time: 1955
File: "/content/input/flower.bmp"(time: 34373)
  Load time: 567
  Gauss time: 20533
  Sobel time: 12232
  Store time: 1041
File: "/content/input/bath.bmp"(time: 26116)
  Load time: 388
  Gauss time: 15095
  Sobel time: 9885
  Store time: 748
File: "/content/input/pyramid.bmp"(time: 67645)
  Load time: 847
  